In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from torchvision import datasets, transforms

from nets import NetOneLayer, NetOneLayerLowRank

In [3]:
batch_size = 128
batch_size_test = 1000

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size_test, shuffle=True)

In [5]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
lr = 0.01
momentum = 0.9
n_epochs = 50

# model = NetOneLayerLowRank(n_hidden=2**8, d=2, K=2)
model = NetOneLayer(n_hidden=2**8)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(n_epochs):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.338593
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.331945
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.313817
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.243971
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.216885

Test set: Average loss: -0.2158, Accuracy: 2173/10000 (22%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.193237
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.206975
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.245503
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.244076
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.213951

Test set: Average loss: -0.2686, Accuracy: 2702/10000 (27%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.180673
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.222097
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.085020
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.140434
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.144905

Test set: Average loss: -0.2924, Accuracy: 2938/10000 (29%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.140555
Train Epoch: 3 [12800/60000 (21%)]